In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import copy
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import r2_score, roc_auc_score

In [2]:
train = pd.read_csv("cleaned_million.csv")

In [3]:
drop_list = ['artist_location', 'artist_name', 'artist_id', 'artist_terms', 'artist_terms_freq', 'beats_start', 'artist_terms_weight', 'bars_confidence','bars_start', 'beats_confidence', 'release', 'sections_confidence', 'sections_start', 'segments_confidence', 'segments_loudness_max', 'segments_loudness_max_time', 'segments_loudness_start', 'segments_pitches','segments_start', 'segments_timbre', 'tatums_confidence', 'tatums_start', 'title', 'year']
train = train.drop(drop_list, axis=1)

In [4]:
train.head()

Unnamed: 0  artist_familiarity  artist_hotttnesss  artist_latitude  \
0           1            0.626958           0.434860              NaN   
1           2            0.425724           0.000000              NaN   
2           3            0.611495           0.334520              NaN   
3           4            0.367255           0.311616              NaN   
4           5            0.601306           0.363676         36.87652   

   artist_longitude   duration  end_of_fade_in   key  key_confidence  \
0               NaN  252.99546           0.514   1.0           0.429   
1               NaN   78.02730           0.974   3.0           0.000   
2               NaN  163.63057           0.000   7.0           0.380   
3               NaN  199.99302           0.000  10.0           0.551   
4         -89.58828  279.35302           0.433   9.0           0.519   

   loudness  mode  mode_confidence  song_hotttnesss  start_of_fade_out  \
0   -11.061   0.0            0.670         0.475638            241.424   
1   -24.140   1.0            0.000              NaN             78.027   
2    -5.795   1.0            0.385              NaN            158.511   
3   -16.477   1.0            0.528              NaN            195.808   
4   -12.474   1.0            0.491         0.445455            261.288   

     tempo  time_signature  time_signature_confidence  
0   80.084             4.0                      0.533  
1   54.874             4.0                      0.000  
2   77.150             3.0                      0.369  
3  120.382             4.0                      1.000  
4   99.024             4.0                      1.000

In [4]:
train = train.dropna(axis=0, how='any')

In [6]:
Y = copy.deepcopy(train.song_hotttnesss)
Y.shape

(2209,)

In [7]:
train = train.drop("song_hotttnesss", axis=1)

In [8]:
print(train.shape, Y.shape)

(2209, 16) (2209,)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(train, Y, test_size=0.33, random_state=7)
model = XGBRegressor()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
accuracy = model.score(X_test, y_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 28.88%


In [23]:
X = train.drop(["song_hotttnesss"], axis=1)
Y = train["song_hotttnesss"]
X, X_Val, Y, Y_Val = train_test_split(X, Y)

# A parameter grid for XGBoost
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1) 

grid = GridSearchCV(xgb, params)
grid.fit(X, Y)

# Print the r2 score
print(r2_score(Y_Val, grid.best_estimator_.predict(X_Val))) 

0.32714122739196894


In [25]:
grid.best_params_

{'colsample_bytree': 0.9,
 'gamma': 0.4,
 'max_depth': 4,
 'min_child_weight': 4,
 'subsample': 0.9}

In [34]:
X = train.drop(["song_hotttnesss"], axis=1)
Y = train["song_hotttnesss"]
X, X_Val, Y, Y_Val = train_test_split(X, Y)

# A parameter grid for XGBoost
params = {'learning_rate':[0.05, 0.1]}

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=-1, min_child_weight=4, gamma=0.4,  subsample=0.9,
colsample_bytree=0.9, max_depth=4, n_estimators=950) 

grid = GridSearchCV(xgb, params)
grid.fit(X, Y)

# Print the r2 score
print(r2_score(Y_Val, grid.best_estimator_.predict(X_Val))) 

0.3775282582022438


In [30]:
grid.best_params_

{'learning_rate': 0.05, 'n_estimators': 800}